# The Perceptron Learning Algorithm

> In this problem, you will create your own target function f and data set D to see
how the Perceptron Learning Algorithm works. Take d = 2 so you can visualize the
problem, and assume X = [−1, 1] × [−1, 1] with uniform probability of picking each
x ∈ X .

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
import random
import math
from functools import reduce

## Create the experiment

> In each run, **choose a random line in the plane as your target function f** (do this by
taking two random, uniformly distributed points in [−1, 1] × [−1, 1] and taking the
line passing through them), where one side of the line maps to +1 and the other maps
to −1. **Choose the inputs xn of the data set as random points (uniformly in X ), and
evaluate the target function on each xn to get the corresponding output yn.**

In [2]:
# Create a dataset of N training points, evaluated with the target function f
def create_dataset(f, N):
    dataset = [];
    
    for _ in range(N):
        # Choose the inputs xn of the data set as random points
        x1, x2 = np.random.uniform(-1,1,2)
        
        # evaluate the target function on each xn to get the corresponding output yn.
        y = 1 if x2 > f(x1) else -1

        dataset.append([[x1, x2], y])
    
    return dataset;

# Create an experiment: target function and dataset of N training points
def create_experiment(N):
    # Choose a random line in the plane as your target function f
    x1, y1 = np.random.uniform(-1,1,2)
    x2, y2 = np.random.uniform(-1,1,2)

    a = (y2 - y1)/(x2 - x1)

    b = random.uniform(-1, 1)

    def f(x):
        y = a*x + b
        return y
    
    # Create training dataset by selecting N random points, and evaluating f on them
    dataset = create_dataset(f, N)
        
    return (f, dataset)

## Visualization example of one experimentation

In [3]:
# f is our target function
# dataset is training dataset
f, dataset = create_experiment(100)

plt.axis([-1, 1, -1, 1])

x = np.linspace(-1, 1, 2)
plt.plot(x, f(x))

for data in dataset:
    point = data[0]
    output = data[1]
    plt.plot(point[0], point[1],  'go' if output > 0 else 'ro')

## Run the Perceptron Learning Algorithm

In [4]:
def is_misclassified_point(w, data):
    return (np.sign(np.dot(w, [1, data[0][0], data[0][1]])) != data[1])

In [5]:
# In order to get a reliable estimate for these two quantities, you should repeat the experiment for 1000 runs
NUMBER_OF_RUNS = 1000

# N is the number of training points used in the experiment
def repeat_experiment(N):
    experiments = []
    
    for _ in range(NUMBER_OF_RUNS):
        f, dataset = create_experiment(N)

        #  Start the PLA with the weight vector w being all zeros
        w = [0,0,0]

        # consider sign(0) = 0, so all points are initially misclassified
        misclassified_points = dataset

        number_of_iterations = 0
        while len(misclassified_points) > 0:
            # Pick a random misclassified point
            data = random.sample(misclassified_points, 1)[0]

            # Apply PLA iteration
            w = w + np.dot(data[1], [1, data[0][0], data[0][1]])

            misclassified_points = list(filter(lambda data: is_misclassified_point(w, data), dataset))
            number_of_iterations += 1

        # Disagreement probability between g and f
        # approximate it by generating a sufficiently large, separate set of points to estimate it.
        SEPARATED_DATASET_SIZE = 1000
        separated_dataset = create_dataset(f, SEPARATED_DATASET_SIZE)

        new_misclassified_points = list(filter(lambda data: is_misclassified_point(w, data), separated_dataset))

        error_rate = len(new_misclassified_points)/len(separated_dataset)

        experiments.append([number_of_iterations, error_rate])
        
    return experiments
    
def avg_number_of_iterations(experiments):
    return reduce(lambda accumulator, s: accumulator + s[0], experiments, 0)/len(experiments)

def avg_error_rate(experiments):
    return reduce(lambda accumulator, s: accumulator + s[1], experiments, 0)/len(experiments)

### For N = 10 training points:

In [6]:
experiments = repeat_experiment(10)

print(f'Average number of iterations is: {avg_number_of_iterations(experiments)}')
print(f'Average disagreement is: {avg_error_rate(experiments)}')

Average number of iterations is: 9.28
Average disagreement is: 0.10337300000000012


### For N = 100 training points:

In [ ]:
experiments = repeat_experiment(100)

print(f'Average number of iterations is: {avg_number_of_iterations(experiments)}')
print(f'Average disagreement is: {avg_error_rate(experiments)}')

## Use Linear Regression first

In [46]:
f, dataset = create_experiment(100)

In [47]:
def thresholded(x):
    x.insert(0,1)
    return x
    
X = np.array(list(map(lambda x: thresholded(x[0]), dataset)))
print(X)
y = np.array(list(map(lambda x: x[1], dataset)))
print(y)

[[ 1.         -0.31576984 -0.37511253]
 [ 1.          0.09358237  0.27473317]
 [ 1.         -0.08540376  0.16692552]
 [ 1.         -0.83213634  0.93558302]
 [ 1.          0.43278339  0.05815083]
 [ 1.         -0.26014376 -0.19278159]
 [ 1.          0.8177483   0.26188687]
 [ 1.         -0.10321447  0.80434604]
 [ 1.         -0.99668008  0.16055066]
 [ 1.          0.5617159  -0.61916247]
 [ 1.          0.89763049 -0.03360271]
 [ 1.         -0.08471637  0.57284209]
 [ 1.          0.45760668  0.21007361]
 [ 1.         -0.53948754  0.07806272]
 [ 1.         -0.43501745 -0.76009567]
 [ 1.          0.70451636  0.52956268]
 [ 1.         -0.41296907  0.15612094]
 [ 1.          0.31817222 -0.06714057]
 [ 1.          0.4662288  -0.35962517]
 [ 1.          0.55207052 -0.87797809]
 [ 1.          0.29555383  0.96562527]
 [ 1.          0.2171732  -0.23106638]
 [ 1.         -0.20328277 -0.59882927]
 [ 1.          0.1586351  -0.85275347]
 [ 1.          0.89081563 -0.38519073]
 [ 1.         -0.75721233

In [49]:
# Using pseudo-inverse
pseudo_inverse = np.dot(np.linalg.inv(np.dot(X.transpose(), X)), X.transpose())

w = np.dot(pseudo_inverse, y)
print(w)

[ 0.62153059 -0.84688148  0.56664808]


In [55]:
## Using Scikit
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)

w = reg.coef_
w[0] = reg.intercept_
print(w)

[ 0.62153059 -0.84688148  0.56664808]
